In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler, Sampler
from torch.utils.data import DataLoader, random_split

from metrics import multiclass_accuracy

from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [2]:
# Используем GPU чтобы ускорить вычисления
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f'using device: {device}')

using device: cuda


In [25]:
def train_model(model, train_loader, val_loader, loss, optimizer, scheduler=None, num_epochs=5):    
    
    device = ("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            prediction = model(x)    
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        if scheduler != None:
            scheduler.step()
        
        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval() # Evaluation mode
    #       Implement the inference of the model on all of the batches from loader,
    #       and compute the overall accuracy.
    # Hint: PyTorch has the argmax function!
    correct = 0
    all = 0
    # all_accuracy = []
    for (x, y) in loader:
        x = x.to(device)
        y = y.to(device)
        prob = model(x)
        predict = torch.argmax(prob, dim=1)
        correct += torch.sum(predict == y)
        all += len(predict)
        # accuracy = multiclass_accuracy(prob, y)
        # all_accuracy.append(accuracy)
        
    return float(correct) / float(all)

In [26]:
data_train = dset.SVHN('data/', download=True, split='train',
                       transform=transforms.Compose([transforms.ToTensor()])
                      )
data_test = dset.SVHN('data/', download=True, split='test', 
                      transform=transforms.Compose([transforms.ToTensor()]))

batch_size = 128

test_size = int(len(data_test) * 0.3)
val_size = len(data_test) - test_size

test_data, val_data = random_split(data_test, [test_size, val_size])

train_loader = DataLoader(data_train, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [38]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(3*32*32, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(), 
                        
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            
            nn.Linear(100, 10)
            )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
        

In [52]:
model = NeuralNetwork().to(device=device)

# optimizer = optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0.1)
# optimizer = optim.Adam(model.parameters(), lr=0.0002)
optimizer = optim.Adam(model.parameters(), lr=0.00002)
# optimizer = optim.SGD(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss().type(torch.FloatTensor)

In [54]:
optimizer = optim.Adam(model.parameters(), lr=0.00002)

loss_history, train_history, val_history = train_model(model, train_loader, val_loader, loss, optimizer, scheduler=None, num_epochs=25)

cuda
Average loss: 0.706993, Train accuracy: 0.797890, Val accuracy: 0.738956
Average loss: 0.684348, Train accuracy: 0.803596, Val accuracy: 0.743676
Average loss: 0.664970, Train accuracy: 0.808114, Val accuracy: 0.750316
Average loss: 0.646891, Train accuracy: 0.812769, Val accuracy: 0.748505
Average loss: 0.631059, Train accuracy: 0.816768, Val accuracy: 0.750206
Average loss: 0.616008, Train accuracy: 0.820631, Val accuracy: 0.751139
Average loss: 0.602074, Train accuracy: 0.824153, Val accuracy: 0.757340
Average loss: 0.589347, Train accuracy: 0.827593, Val accuracy: 0.757888
Average loss: 0.577766, Train accuracy: 0.831115, Val accuracy: 0.760907
Average loss: 0.566755, Train accuracy: 0.834473, Val accuracy: 0.758657
Average loss: 0.556658, Train accuracy: 0.836548, Val accuracy: 0.763815
Average loss: 0.546932, Train accuracy: 0.839851, Val accuracy: 0.767931
Average loss: 0.538305, Train accuracy: 0.842295, Val accuracy: 0.767601
Average loss: 0.530142, Train accuracy: 0.8448

In [55]:
# Как всегда, в конце проверяем на test set
test_accuracy = compute_accuracy(model, test_loader)
print("Test accuracy: %2.4f" % test_accuracy)

Test accuracy: 0.7558
